In [ ]:
!pip install fastText
!pip install datasets

In [10]:
import fasttext
from datasets import load_dataset

dataset_train = load_dataset('imdb', split='train')

Reusing dataset imdb (/home/victor/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


In [11]:
import random
random.seed(42)

In [19]:
from string import punctuation


def to_lower(review):
	return str.lower(review)


def no_punct(review):
	return review.translate(str.maketrans('', '', punctuation))


def to_fasttext(review, label):

	label_str = 'positive'
	if label == 0:
		label_str = 'negative'
	return '__label__' + label_str + " " + review


def pretreatment(dataset):
	labels = dataset['label']
	reviews = dataset['text']
	corrected_review_list = []
	for i in range(len(reviews)):
		lower = to_lower(reviews[i])
		no_punc = no_punct(lower)
		corrected_review = to_fasttext(no_punc, labels[i])
		corrected_review_list.append(corrected_review)
	return corrected_review_list


X_treated = pretreatment(dataset_train)
random.shuffle(X_treated)
with open("X_train.txt", mode="w") as file:
	file.write("\n".join(X_treated))

In [20]:
model = fasttext.train_supervised(input="X_train.txt")

Read 5M words
Number of words:  121891
Number of labels: 2
Progress: 100.0% words/sec/thread: 1605105 lr:  0.000000 avg.loss:  0.401958 ETA:   0h 0m 0s


In [21]:
dataset_test = load_dataset("imdb", split="test")
X_test_treated = pretreatment(dataset_test)
random.shuffle(X_test_treated)
X_test_treated[:5]

Reusing dataset imdb (/home/victor/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


['__label__positive the pickle was the most underrated film of the decade despite my best efforts at home it is still seen as a bad movie i say to hell with everyone on this every one doesnt understand that it is in part a comedy with a dramatic twist at the end danny aiello plays a burnt out director with perfection and conveys a sad depressed man the scene when he is trying to record a last message is quite good the interludes of stones movie are absolutely funny this is harry stone and today i become a man in all a fine and sadly misunderstood movie plus a great cameo by little richard hilarious i hope that more people see this movie because 202 people is not enough for a consensusbr br silence dogood',
 '__label__positive i really liked this movie i watched it last night on the public broadcasting system the part i liked about it was the fact that they dealt with issues of today not in the future or the past they basically had some terrorists take a van or two and rent them out to 

In [22]:
with open("X_test.txt", mode="w") as file:
	file.write("\n".join(X_test_treated))
model.test("X_test.txt")


(25000, 0.87552, 0.87552)

In [16]:
model.test("X_train.txt")

(25000, 0.90168, 0.90168)

In [ ]:
fasttext.version

In [ ]:
!pip ver